In [165]:
import subprocess
from pandas import read_csv
from random import sample
from pandas import read_excel
import geopandas as gpd
import numpy as np
import json

# Import 

In [166]:
CRS_LAT_LON = "epsg:4326"
# CRS_WGS84 = "EPSG:32719"
CRS_WGS84 = "EPSG:3857"
CRS_CHILE19S = "EPSG:20049"
# geo_data_proy[geo_data_proy["RBD"]==913163]['closest'].values[0]

In [167]:
col_types = {"RBD": np.int32, "UTE": np.int32, "Region": str, "Nombre": str, "Direccion": str, "Comuna": str, "Provincia": str, "closest": list}
data = read_excel("../../Data/Colegios(Continental)2020.xlsx", dtype=col_types)

data['closest'] = data['closest'].apply(json.loads)
data['value'] = 0

In [168]:
geometry = gpd.points_from_xy(data['Longitud'], data['Latitud'], crs=CRS_LAT_LON)
geo_data = gpd.GeoDataFrame(data.drop(columns=['Latitud', 'Longitud']), geometry=geometry, crs=CRS_LAT_LON)

In [169]:
geo_data_proy = geo_data.to_crs(CRS_CHILE19S)

In [170]:
geo_data_proy["x"] = geo_data_proy.geometry.x
geo_data_proy["y"] = geo_data_proy.geometry.y

# Algorithm

In [171]:
# compile_command = ["g++", "TSP.cpp", "-o", "TSP"]
# run_command = ["./TSP"]
# compile_process = subprocess.run(compile_command, stdout = subprocess.PIPE)


In [172]:
MAXN = 12 #Máxima cantidad de colegios
#MAXR = 5000 #Máxima cantidad de raciones
MAXD = 1006065184946015 #Máxima distancia entre el colegio de inicio
#C = 600000 #Costo fijo de tener una furgoneta
K = 10 #Número de ejemplos por cada colegio
#factor = (10**10) #Factor de conversión a pesos

In [197]:
for index, row in geo_data_proy.iterrows():
    options = [el for el in row['closest'] if el != row['RBD']]
    dist_options = [0 for _ in range(len(options))]
    min_dist = float("Inf")
    for j in range(len(options)):
        row_options = geo_data_proy[geo_data_proy['RBD']==options[j]]
        dist_j = abs(row_options["x"] - row["x"]).values[0] + abs(row_options["y"] - row["y"]).values[0]
        dist_options[j] = dist_j
        if min_dist > dist_j and dist_j != 0:
            min_dist = dist_j
    
    prob_options = [1/i if i!=0 else min_dist/10 for i in dist_options] # Existen colegios que están literalmente en el mismo lugar raaaro
    sum_dist = sum(prob_options)
    for j in range(len(options)):
        prob_options[j] = prob_options[j]/sum_dist
    suma = 0
    cant = 0
    size_i = min(MAXN, len(options)) # A veces la cant de colegios no alcanza para MAXN
    # print(dist_options)
    for j in range(K):
        # print(row['RBD'], options)
        # print(dist_options)
        sample_i = np.random.choice(options, size=size_i, replace=False, p=prob_options)
        # print(sample_i)
        sample_i = [row['RBD']] + sample_i

        entrada = str(len(geo_data_proy)) + "\n"
        for u in sample_i:
            entrada = entrada + str(row["x"]) + " " + str(row["y"]) + "\n"
        
        # run_process = subprocess.run(run_command, input = entrada.encode(), stdout = subprocess.PIPE)
        #res = (int(run_process.stdout.decode()) + C)/len(schools)
        # res = int(run_process.stdout.decode())/len(sample_i)
        res = 5 # 9m12s

        for u in sample_i:
            suma += res
            cant += 1
    row['value'] = suma/cant




In [ ]:
geo_data_proy.to_csv("colegios_final.csv")
